# Customer Churn Prediction: from Python to Modeler and back to Python


Adapted from original by Heba El-Shimy https://raw.githubusercontent.com/IBM/customer-churn-prediction/master/notebooks/customer-churn-prediction.ipynb 

-------------------

- This article: https://medium.com/@markryan_69718/watson-studio-desktop-first-impressions-5a85309597d0 describes adapting the original Python-based churn solution described here: https://developer.ibm.com/patterns/predict-customer-churn-using-watson-studio-and-jupyter-notebooks/
- the article describes a simple Modeler flow https://github.com/ryanmark1867/shared_ml/blob/master/churn%20flow%20Feb%202019.str?raw=true that implements a subset of the original Python-based churn solution
- this notebook takes things full circle by attempting to implement the Modeler flow "note for note" back into a much simplified Python notebook that captures nuances of the Modeler flow

This notebook shows screenshots of the Modeler flow followed by the Python that attempts to implement the same function.


# Load Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
import sklearn.feature_selection
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn import metrics
from scipy import stats
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
import warnings
warnings.filterwarnings('ignore')


testproportion = 0.3


# The Dataset

The same dataset used in this notebook was used in the Modeler flow described above: https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Telco-Customer-Churn.csv


# Modeler Flow: Ingest dataset
- the first step in the Modeler flow is to ingest the dataset

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_ingest_data.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>


# Python: Ingest Dataset


In [2]:
url="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/WA_Fn-UseC_-Telco-Customer-Churn.csv"
customer_data = pd.read_csv(url)
customer_data.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Modeler Flow: Select subset of columns from original dataset
- the next node of the Modeler flow selects a subset of the columns from the original dataset

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_filter_selected.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Select subset of columns from original dataset

In [3]:
# define the columns that we are going to keep
retain_columns = ['MonthlyCharges','TotalCharges','InternetService','PaymentMethod','OnlineSecurity','Churn','Contract','tenure']

In [4]:
# select a subset of columns from the original dataset
customer_data = customer_data[['MonthlyCharges','TotalCharges','InternetService','PaymentMethod','OnlineSecurity','Churn','Contract','tenure']]
customer_data.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,29.85,29.85,DSL,Electronic check,No,No,Month-to-month,1
1,56.95,1889.50,DSL,Mailed check,Yes,No,One year,34
2,53.85,108.15,DSL,Mailed check,Yes,Yes,Month-to-month,2
3,42.30,1840.75,DSL,Bank transfer (automatic),Yes,No,One year,45
4,70.70,151.65,Fiber optic,Electronic check,No,Yes,Month-to-month,2


# Modeler flow: Fill empty values in TotalCharges column
- in the Modeler flow we only fill the empty value in TotalCharges - do the same in Python

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_filler.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Fill empty values in TotalCharges column

In [5]:
# check the number of empty values in TotalCharges before and after filling the missing values
print("empty values in TotalCharges before filling:",customer_data['TotalCharges'].isnull().sum())
customer_data['TotalCharges'].fillna(value=0, inplace=True)
print("empty values in TotalCharges after filling:",customer_data['TotalCharges'].isnull().sum())

empty values in TotalCharges before filling: 11
empty values in TotalCharges after filling: 0


# Modeler flow: Prep data
The next step in the Modeler flow incorporates many preparation steps, including:
1. replacing categorical tokens with numerical IDs
2. scaling continuous values
3. replace outliers (values that are beyond a threshold) with the threshold

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Prep data 1 - replace categorical tokens with numerical IDs

In [6]:
# identify the subsets of columns that are categorical and continuous
categorical_columns = ['InternetService', 'PaymentMethod', 'OnlineSecurity', 'Churn', 'Contract']
continuous_columns = ['MonthlyCharges', 'TotalCharges','tenure']

In [7]:
# input dataframe and list of colums to be zscore scaled' return dataframe with those columns scaled
def scale_columns(df,col_list):
    for col in col_list:
        print("col is",col)
        df[col] = df[col].apply(zscore)
    return df

In [8]:
# input dataframe and list of columns to be encoded; return dataframe with those columns encoded
def encode_columns(df,col_list):
    for col in col_list:
        print("col is",col)
        le = LabelEncoder()
        le.fit(df[col].tolist())
        df[col] = le.transform(df[col])
    return(df)
 

In [9]:
# replace tokens in categorical columns with numeric IDs
customer_data = encode_columns(customer_data,categorical_columns)
customer_data.head()

col is InternetService
col is PaymentMethod
col is OnlineSecurity
col is Churn
col is Contract


,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,29.85,29.85,0,2,0,0,0,1
1,56.95,1889.50,0,3,2,0,1,34
2,53.85,108.15,0,3,2,1,0,2
3,42.30,1840.75,0,0,2,0,1,45
4,70.70,151.65,1,2,0,1,0,2


In [10]:
# print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())

min MonthlyCharges 18.25
max TotalCharges 8684.8
min TotalCharges 0.0
max tenure 72
min tenure 0


# Modeler flow: Prep data 2 - scale continuous values


<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep_scaling.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Prep data 2 - scale continuous values

In [11]:
# scale continuous columns using zscore
customer_data[continuous_columns] = customer_data[continuous_columns].apply(zscore)

# Modeler flow: Prep data 3 - replace outliers
- replace outliers (values that are beyond a threshold) with the threshold

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep_outliers_replace.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Prep data 3 - replace outliers

In [12]:
def sd_max(x,sd,multiplier):
    if x > multiplier*sd:
        return multiplier*sd
    else:
        return x

def replace_outliers(df,multiplier):
    for col in continuous_columns:
        sd = df.loc[:,col].std()
        print("sd",sd)
        df[col] = df[col].apply(sd_max,args=(sd,multiplier))
    return df
        
        

In [13]:
# replace outliers that are more than a boundary with the boundary value
customer_data = replace_outliers(customer_data,3.0)

sd 1.0000710000355904
sd 1.0000710000355884
sd 1.0000710000355943


In [14]:
print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())

max MonthlyCharges 1.79435215026
min MonthlyCharges -1.54585982007
max TotalCharges 2.82580557787
min TotalCharges -1.00577983371
max tenure 1.61370124044
min tenure -1.3181649474


In [15]:
customer_data.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,-1.160323,-0.992611,0,2,0,0,0,-1.277445
1,-0.259629,-0.172165,0,3,2,0,1,0.066327
2,-0.362660,-0.958066,0,3,2,1,0,-1.236724
3,-0.746535,-0.193672,0,0,2,0,1,0.514251
4,0.197365,-0.938874,1,2,0,1,0,-1.236724


In [16]:
print("max MonthlyCharges", customer_data['MonthlyCharges'].max())
print("min MonthlyCharges", customer_data['MonthlyCharges'].min())
print("max TotalCharges", customer_data['TotalCharges'].max())
print("min TotalCharges", customer_data['TotalCharges'].min())
print("max tenure", customer_data['tenure'].max())
print("min tenure", customer_data['tenure'].min())


max MonthlyCharges 1.79435215026
min MonthlyCharges -1.54585982007
max TotalCharges 2.82580557787
min TotalCharges -1.00577983371
max tenure 1.61370124044
min tenure -1.3181649474


# Modeler flow: data post Auto Data Prep
- values in categorical columns have been replaced with numerical IDs
- the continuous columns have been scaled with a zscore transformation and outliers have been replaced

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_auto_data_prep_preview_post_adp_v2.jpg" width="950" alt="Icon"> </th>
   </tr>
</table>

# Python: data post data prep

In [17]:
customer_data.head(10)

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Churn,Contract,tenure
0,-1.160323,-0.992611,0,2,0,0,0,-1.277445
1,-0.259629,-0.172165,0,3,2,0,1,0.066327
2,-0.362660,-0.958066,0,3,2,1,0,-1.236724
3,-0.746535,-0.193672,0,0,2,0,1,0.514251
4,0.197365,-0.938874,1,2,0,1,0,-1.236724
5,1.159546,-0.643789,1,2,0,1,0,-0.992402
6,0.808907,-0.145738,1,1,0,0,0,-0.422317
7,-1.163647,-0.872587,0,3,2,0,0,-0.910961
8,1.330711,0.338085,1,2,0,1,0,-0.177995
9,-0.286218,0.533044,0,0,2,0,1,1.206498


# Modeler flow: Split dataset into test and train
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_partition.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Split dataset into test and train

In [18]:
# define label
y_le = customer_data['Churn']
# define input values
X_selected = customer_data.drop(['Churn'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_le,\
                                                    test_size=testproportion, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4930, 7) (4930,)
(2113, 7) (2113,)


In [19]:
X_test.head()

,MonthlyCharges,TotalCharges,InternetService,PaymentMethod,OnlineSecurity,Contract,tenure
185,-1.328164,-0.994838,0,2,0,0,-1.277445
2715,-1.313208,-0.566163,2,0,1,0,0.351370
3825,-1.509300,-0.550611,2,3,1,2,0.799294
1807,0.385148,-0.972096,1,2,0,0,-1.277445
132,-0.472339,0.432521,0,0,0,2,1.410099


# Modeler flow: Train Support Vector Machine
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_SVM_train.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Train Support Vector Machine

In [20]:
# fit SVM using training data
clf_svc = svm.SVC(random_state=42)
clf_svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

# Modeler flow: Train Logistic Regression
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_logistic_regression_train.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Train Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
model = clf_lr.fit(X_train, y_train)
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Modeler flow: Evaluate Support Vector Machine
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_SVM_results2.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Evaluate Support Vector Machine

In [22]:
# Get model confidence of predictions
y_score_svc = clf_svc.decision_function(X_test)
y_score_svc

array([-0.06117747, -1.01810821, -1.01249794, ..., -1.01551142,
       -0.02449712, -1.04244246])

# Modeler flow: SVM results

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_SVM_results_text.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: SVM results

In [23]:
# Get accuracy score
y_pred_svc = clf_svc.predict(X_test)
acc_svc = accuracy_score(y_test, y_pred_svc)
print(acc_svc)

0.795551348793


In [24]:
# Get Precision vs. Recall score
average_precision_svc = average_precision_score(y_test, y_score_svc)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision_svc))

Average precision-recall score: 0.64


# Modeler flow: Evaluate Logistic Regression
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_logistic_regression_results.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>



# Python: Evaluate Logistic Regression

In [25]:
y_score_lr = clf_lr.decision_function(X_test)
y_score_lr

array([-0.55997081, -2.6605672 , -5.07462064, ..., -4.51331337,
        0.30978154, -0.71314255])

# Modeler flow: Logistic Regression results

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/june2019_ML_bootcamp/master/flow_logistic_regression_results_text.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Python: Logistic Regression results

In [26]:
y_pred_lr = clf_lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(acc_lr)

0.793658305726


In [27]:
average_precision_lr = average_precision_score(y_test, y_score_lr)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision_lr))

Average precision-recall score: 0.64
